Форматы записи меток периодов основного тона:

1. .seg_G1

In [ ]:
!wget https://pkholyavin.github.io/mastersprogramming/cta0001.seg_G1

In [1]:
from itertools import product
letters = "GBRY"
nums = "1234"
levels = [ch + num for num, ch in product(nums, letters)]
level_codes = [2 ** i for i in range(len(levels))]
code_to_level = {i: j for i, j in zip(level_codes, levels)}
level_to_code = {j: i for i, j in zip(level_codes, levels)}

In [2]:
def read_seg(filename: str, encoding: str = "utf-8-sig") -> tuple[dict, list[dict]]:
    with open(filename, encoding=encoding) as f:
        lines = [line.strip() for line in f.readlines()]

    # найдём границы секций в списке строк:
    header_start = lines.index("[PARAMETERS]") + 1
    data_start = lines.index("[LABELS]") + 1

    # прочитаем параметры
    params = {}
    for line in lines[header_start:data_start - 1]:
        key, value = line.split("=")
        params[key] = int(value)

    # прочитаем метки
    labels = []
    for line in lines[data_start:]:
        # если в строке нет запятых, значит, это не метка и метки закончились
        if line.count(",") < 2:
            break
        pos, level, name = line.split(",", maxsplit=2)
        label = {
            "position": int(pos) // params["BYTE_PER_SAMPLE"] // params["N_CHANNEL"],
            "level": code_to_level[int(level)],
            "name": name
        }
        labels.append(label)
    return params, labels

In [ ]:
params, labels = read_seg("cta0001.seg_G1")
labels

На уровне G1 часто (но не всегда!) также находятся метки, обозначающие границы файла. Они не являются метками границ периодов ОТ!

In [ ]:
pitch_labels = labels[1:-1]

Вопрос:
1. Как, зная время начала и конца периода, определить его длину?
2. Как, зная длину периода, определить значение частоты основного тона в этот момент времени?

Не любая пара меток в файле соответствует периоду ОТ. Некоторые из них отмечают глухие участки:

<img src="https://pkholyavin.github.io/mastersprogramming/voiceless_part.png" width="1000">